## Objetivo do Projeto: Game Industry Lakehouse
Este projeto visa desenvolver um pipeline de dados ponta a ponta para analisar a correlação entre crítica, precificação e comportamento do consumidor na indústria de videogames.

O projeto visa ir além da análise exploratória básica, focando na implementação de uma arquitetura Lakehouse no Databricks, para melhor governança, rastreabilidade e automação de ifnraestrutura.

## Pilares Estratégicos e Técnicos
1. Engenharia de Dados & Arquitetura Medalhão
  Ingestão e Unificação: Consolidação de fontes heterogêneas (dados históricos de vendas de consoles vs. dados digitais da Steam/PC) em uma camada Bronze.

    Tratamento (**Silver**): Limpeza de dados, padronização de nomenclatura e remoção de incosnsistências para garantir a qualidade da informação.

    Modelagem (**Gold**): Criação de uma visão unificada (Unified_Games_Analytics) utilizando estratégias de Join para integrar as informações de forma considtente.

2. Data Ops & Governança (Infrastructure as Code)

    Unity Catalog: Organização hierárquica rigorosa (Catalog > Schema > Table) para controle de acesso e organização lógica.

    Integração Git & Versionamento: Implementação de rotinas de "Engenharia Reversa" para extração automatizada de DDLs, permitindo que o esquema do banco de dados seja versionado no GitHub.

3. Analytics & Inteligência de Negócio

    Responder a perguntas chave de negócio:

    1. O sucesso da crítica dita o tamanho do desconto? Jogos com maior nota possuem maior preço?
    2. O sucesso das notas dos jogadores dita o tamanho do desconto? Jogos com maior nota possuem maior preço?
    3. Quais gêneros possuem a melhor nota média?
    4. Quais generos que possuem maior desconto?
    5. A nota da crítica é muito diferente da nota dos jogadores?

# Coleta
Os dados foram obtidos através da plataforma Kaggle, através dos seguintes Links:
- Steam - https://www.kaggle.com/datasets/alyahmedts13/top-pc-games-metacritic-vs-steam-popularity
- Game Sales - https://www.kaggle.com/datasets/rush4ratio/video-game-sales-with-ratings

## Carga
A estratégia de coleta para este MVP foi a ingestão em Batch de arquivos estáticos diretamente para o armazenamento do Databricks, simulando o recebimento de dados de parceiros externos.

## Confidencialidade
Como são dados públicos de jogos, os dados não contêm PII de usuários reais, logo não apresentam risco de compromentimento de confidencialidade.

# Modelagem
Para a modelagem de dados, foi adotado o padrão de medalhão, onde:
- Bronze: Refere-se à camada de dados "crua" sem nenhum tratamento inicial
- Silver: Refere-se à camdada de dados tratada


# Criação do ambiente no git
O trecho abaixo é responsável por criar um repositório no GitHub e fazer o upload das bases de dados que serão utilizadas nas consultas.

In [0]:
import os

# Configuração
CATALOG_NAME = "schemas"
# Caminho da pasta do Repo
base_path = os.getcwd()
REPO_PATH = os.path.join(base_path, "ddl_export")

print(f"Os arquivos serão salvos em: {REPO_PATH}")

# Cria a pasta de exportação
os.makedirs(REPO_PATH, exist_ok=True)

# Lista todos os schemas (databases) do catálogo
schemas = spark.sql(f"SHOW SCHEMAS IN {CATALOG_NAME}").collect()

for schema_row in schemas:
    schema_name = schema_row['databaseName']
    if schema_name == 'information_schema': continue # Pula sistema
    
    print(f"Processando Schema: {schema_name}...")
    
    # Lista tabelas do schema
    tables = spark.sql(f"SHOW TABLES IN {CATALOG_NAME}.{schema_name}").collect()
    
    for table_row in tables:
        table_name = table_row['tableName']
        full_table_name = f"{CATALOG_NAME}.{schema_name}.{table_name}"
        
        try:
            # Pega o DDL (Comando CREATE TABLE)
            ddl_df = spark.sql(f"SHOW CREATE TABLE {full_table_name}")
            ddl_content = ddl_df.first()[0]
            
            # Salva em arquivo .sql
            filename = f"{schema_name}_{table_name}.sql"
            file_path = os.path.join(REPO_PATH, filename)
            
            with open(file_path, "w") as f:
                f.write(ddl_content)
                
        except Exception as e:
            print(f"Erro ao exportar {full_table_name}: {str(e)}")

print(f"Exportação concluída em: {REPO_PATH}")

# Análise das bases cruas

Incialmente, determinarei quais os campos de cada uma das tabelas serão utilizados:

**Tabela Steam_Spy_Data:**
- **name**: Nome do jogo  
- **developer**: Desenvolvedor do jogo  
- **publisher**: Empresa que publicou o jogo  
- **initialprice**: Preço inicial do jogo , em dólares, em forma de inteiro (6900 = $ 69,00)
- **price**: Preço atual do jogo, em dólares, em forma de inteiro (6900 = $ 69,00)
- **discount**: Desconto, em %, aplicado ao jogo em forma de inteiro (6 = 6%)
            
Após a determinação dos campos a serem utilizados, identifico quais campos estão nulos, que poderão impedir a utilização dos dados ou o cruzamento das informações com a tabela **game_sales** de forma efetiva.

**Tabela Game_sales:**
- **Name**: Nome do jogo  
- **Year_of_Release**: Ano de lancamento  
- **Genre**: Genero do jogo  
- **Publisher**: Empresa que publicou o jogo  
- **Critic_Score**: Nota do jogo dada pela critica, podendo variar de 0 a 100  
- **User_Score**: Nota do jogo dada pelos jogadores, podendo variar de 0 a 10   
- **Developer**: Desenvovledora do jogo
            
Após a determinação dos campos a serem utilizados, identifico quais campos estão nulos, que poderão impedir a utilização dos dados ou o cruzamento das informações com a tabela **Steam_Spy_Data** de forma efetiva.

In [0]:
%sql
SELECT
  -- Contagem total de linhas na tabela
  COUNT(*) AS Total_Registros,

  -- Verifica Nulos para cada campo (1 se for nulo, 0 se não for)
  SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) AS Name,
  SUM(CASE WHEN developer IS NULL THEN 1 ELSE 0 END) AS Developer,
  SUM(CASE WHEN publisher IS NULL THEN 1 ELSE 0 END) AS Publisher,
  SUM(CASE WHEN owners IS NULL THEN 1 ELSE 0 END) AS Owners,
  SUM(CASE WHEN initialprice IS NULL THEN 1 ELSE 0 END) AS InitialPrice,
  SUM(CASE WHEN price IS NULL THEN 1 ELSE 0 END) AS ActualPrice,
  SUM(CASE WHEN discount IS NULL THEN 1 ELSE 0 END) AS Discount

FROM schemas.bronze.steam_spy_data;

Inicialmente, percebe-se que existem campos nulos apenas nas colunas Developer e Publisher, de forma que apenas estes registros deverão ser desconsiderados.

In [0]:
%sql
select * from schemas.bronze.game_sales;

SELECT
    COUNT(*) AS Total_Registros,
    SUM(CASE WHEN Name IS NULL THEN 1 ELSE 0 END) AS Name,
    SUM(CASE WHEN Year_of_Release IS NULL THEN 1 ELSE 0 END) AS Year_of_Release,
    SUM(CASE WHEN Genre IS NULL THEN 1 ELSE 0 END) AS Genre,
    SUM(CASE WHEN Publisher IS NULL THEN 1 ELSE 0 END) AS Publisher,
    SUM(CASE WHEN Critic_Score IS NULL THEN 1 ELSE 0 END) AS Critic_Score,
    SUM(CASE WHEN User_Score IS NULL THEN 1 ELSE 0 END) AS User_Score,
    SUM(CASE WHEN Developer IS NULL THEN 1 ELSE 0 END) AS Developer
FROM
    schemas.bronze.game_sales;

Na tabela **game_sales** o numero de registros nulos é ligeiramente maior, de forma que estes campos deverão ser excluídos no momento do tratamento para a camada silver.

In [0]:
%sql
CREATE OR REPLACE TABLE schemas.silver.steam_clean AS
SELECT
  name AS Name,                 -- Nome do jogo
  developer AS Developer,       -- Desenvolvedor do jogo
  publisher AS Publisher,       -- Publicadora do jogo
  initialprice AS InitialPrice, -- Preço inicial do jogo
  price AS ActualPrice,         -- Preço atual do jogo
  discount AS Discount          -- Desconto aplicado ao jogo
FROM schemas.bronze.steam_spy_data
WHERE Developer IS NOT NULL
  AND Publisher IS NOT NULL;

-- Rodando query para verificar se as inserções foram realizadas adequadamente.
SELECT * FROM schemas.silver.steam_clean LIMIT 100;

Tratamento da tabela **steam_spy_data**, criando  uma tabela tratada na camada Silver.

In [0]:
%sql
CREATE OR REPLACE TABLE schemas.silver.game_sales_clean AS
SELECT
    Name,                -- Título do jogo
    CAST(Year_of_Release AS INT) Year_of_Release,     -- Ano de lançamento do jogo (Ignorando os casos em que não está armazenado um valor numérido na data)
    Genre,               -- Gênero do jogo
    Publisher,           -- Empresa responsável pela publicação
    Critic_Score,        -- Nota dada por críticos
    CAST(User_Score AS DOUBLE) AS User_Score,          -- Nota dada por usuários (com existem umas linhas com valor textual, se faz necessário converter em dbl)
    Developer            -- Empresa responsável pelo desenvolvimento
FROM
    schemas.bronze.game_sales
WHERE Name  IS NOT NULL
  AND Genre IS NOT NULL
  AND Critic_Score IS NOT NULL
  AND Developer IS NOT NULL
  AND TRY_CAST(Year_of_Release AS INT) IS NOT NULL
  AND TRY_CAST(User_Score AS DOUBLE) IS NOT NULL; -- Caso não seja possível converter para double, eu descarto

SELECT * FROM schemas.silver.game_sales_clean LIMIT 100; 

Tratamento da tabela **game_sales**, criando  uma tabela tratada na camada Silver.

_Após a criação das novas tabelas, se fez necessário rodar novamente o trecho de código responsável pela criação do ambiente no git e upload de tabelas._

# Unificação das tabelas
Agora com ambas as tabelas previamente tratadas, torna-se possível fazer a unificação das tabelas, de forma a ter uma tabela única de onde será possível fazer todas as consultas necessárias.

In [0]:
%sql
-- Cria a tabela Gold unificada
CREATE OR REPLACE TABLE schemas.gold.unified_games_analytics AS
SELECT
    -- Uso COALESCE para pegar o nome de onde ele existir
    -- Se o jogo existir nas duas, pega o primeiro
    COALESCE(gs.Name, st.Name) AS Game_Name,
    COALESCE(gs.Publisher, st.Publisher) AS Publisher,
    COALESCE(gs.Developer, st.Developer) AS Developer,

    -- Dados da Game Sales (Contexto e Crítica)
    gs.Year_of_Release,
    gs.Genre,
    gs.Critic_Score,
    gs.User_Score,

    -- Dados da Steam (Financeiro)
    st.InitialPrice/100 AS InitialPrice,
    st.ActualPrice/100 AS ActualPrice,
    st.Discount

FROM schemas.silver.game_sales_clean gs

-- Uso o inner p garantir que somente sejam considerados as entradas que contenham 100% das informaçoes desejadas
INNER JOIN schemas.silver.steam_clean st
    -- Normalizao com LOWER e TRIM p evitar que espaços ou maiúsculas impeçam a união
    ON LOWER(TRIM(gs.Name)) = LOWER(TRIM(st.Name));
  

-- Validação dos dados
SELECT * FROM schemas.gold.unified_games_analytics;

Devido a variedade de formas de registro do nome que o mesmo jogo pode ter em ambas as tabelas, realizar a unificação da tabelas resulta em uma diminuição expressiva da quantidade de registros, reduzindo a base de 10k entradas da Steam para apenas 1.069 intercedentes, logo a analise pode ser comprometida.
Podem ocorrer também registros em que jogos diferentes possuem o mesmo registro, de forma a poder distorcer a análise.
A falta de um registro unificado para produtos deste gênero, como o ISBN é para os livros, faz com que o cruzamento de bases de origens distintas seja dificultado nestas análises.

_Após a criação das novas tabelas, se fez necessário rodar novamente o trecho de código responsável pela criação do ambiente no git e upload de tabelas._

In [0]:
%sql
-- Identificar outliers e validar se os valores (Scores e Preços) estão dentro do esperado.
SELECT 
    -- Estatísticas de Notas (Crítica)
    MIN(Critic_Score) as Min_Critic,
    ROUND(AVG(Critic_Score),1) as Media_Critic,
    MAX(Critic_Score) as Max_Critic,
    
    -- Estatísticas de Notas (Usuário)
    MIN(User_Score) as Min_User,
    ROUND(AVG(User_Score),1) as Media_User,
    MAX(User_Score) as Max_User,

    -- Estatísticas de Preço
    MIN(ActualPrice) as Min_Price,
    ROUND(AVG(ActualPrice), 2) as Media_Price,
    MAX(ActualPrice) as Max_Price,
    
    -- Estatísticas de Desconto
    MIN(Discount) as Min_Discount,
    MAX(Discount) as Max_Discount,

    -- Estatísticas ded data de lancamento
    MIN(Year_of_Release) as Min_Year_of_Release,
    MAX(Year_of_Release) as Max_Year_of_Release
    


FROM schemas.gold.unified_games_analytics;

# Qualidade dos Dados
Os resultados validam a consistência e integridade dos dados. 
Observ-se uma **coerência** entre as médias de avaliação (Crítica: 75.6/100 e Usuários: 7.2/10), indicando alinhamento geral de percepção, apesar da ampla dispersão de qualidade (Mín: 27 vs Máx: 97). 
Financeiramente, os dados refletem com precisão a realidade de mercado, abrangendo desde títulos _Free-to-Play_ (0.00) até lançamentos _Premium_ (69.99). A média de preço (~15.95) sugere uma predominância de jogos independentes ou de catálogo na amostra, enquanto o pico de desconto (95%) confirma a captura correta de campanhas promocionais agressivas comuns em plataformas digitais

# Catálogo

## 1. Identificação da Tabela
* **Nome Técnico:** `schemas.gold.unified_games_analytics`
* **Descrição:** Tabela analítica unificada que consolida dados de recepção crítica, popularidade e precificação de jogos eletrônicos.
* **Granularidade:** Um registro por Jogo (Título único).
* **Tipo de Tabela:** Tabela Delta (Gerenciada via Unity Catalog).

---

## 2. Especificação dos Atributos
Descição os campos disponíveis, seus tipos e os domínios dedados identificados durante a fase de perfilamento.

| Coluna | Tipo | Descrição | Valores Observados |
| :--- | :--- | :--- | :--- |
| **`Game_Name`** | `STRING` | Título oficial do jogo (Normalizado). Chave lógica da tabela. | Texto livre. Ex: *"Grand Theft Auto V"* |
| **`Publisher`** | `STRING` | Empresa responsável pela publicação do jogo. | Categorias diversas (EA, Ubisoft, Nintendo, etc.) |
| **`Developer`** | `STRING` | Estúdio responsável pelo desenvolvimento. | Texto livre |
| **`Genre`** | `STRING` | Gênero principal de classificação do jogo. | Ex: Action, Sports, RPG, Shooter |
| **`Year_of_Release`** | `INT` | Ano de lançamento original. | Histórico (Ex: 1980 a 2023) |
| **`Critic_Score`** | `INT` | Nota agregada da crítica especializada (Metacritic). | **Mín: 27** / **Máx: 97** (Escala 0-100) |
| **`User_Score`** | `FLOAT` | Nota agregada de usuários. | **Mín: 1.4** / **Máx: 9.4** (Escala 0-10) |
| **`InitialPrice`** | `FLOAT` | Preço original de lançamento (em USD). | **Mín: 0.00** / **Máx: 69.99** (Escala 0-N)|
| **`ActualPrice`** | `FLOAT` | Preço atual praticado (considerando promoções). | **Mín: 0.00** (Free)  **Média: ~15.95** (Escala 0-N)|
| **`Discount`** | `INT` | Percentual de desconto aplicado no momento da coleta. | **0% a 95%** (Escala 0-100)|
| **`Data_Source`** | `STRING` | Metadado indicando a proveniência do registro (Linhagem). | Categorias: *'Ambas'*, *'Apenas Game Sales'*, *'Apenas Steam'* |

---

## 3. Linhagem e Processamento (Lineage)

### Origem dos Dados
Os dados desta tabela são o resultado do cruzamento entre duas fontes primárias processadas na camada Silver:
1.  **Dados de Vendas Físicas/Consoles:** Provenientes da tabela `schemas.silver.game_sales_clean`.
2.  **Dados Digitais/PC:** Provenientes da tabela `schemas.silver.steam_clean`.


# Solução do problema

## Perguntas determinadas no início do documento:

### 1. O sucesso da crítica dita o tamanho do desconto? Jogos com maior nota possuem maior preço?

In [0]:
%sql
-- Agrupo as notas em faixas para ver se "Nota Alta" = "Desconto baixo"
SELECT 
    CASE 
        WHEN Critic_Score >= 90 THEN '1. Aclamados (90-100)'
        WHEN Critic_Score BETWEEN 75 AND 89 THEN '2. Bons (75-89)'
        WHEN Critic_Score BETWEEN 60 AND 74 THEN '3. Medianos (60-74)'
        WHEN Critic_Score < 60 THEN '4. Fracos (<60)'
        ELSE 'N/A'
    END AS Faixa_Critica,
    COUNT(*) as Qtd_Jogos,
    ROUND(AVG(Discount), 1) as Desconto_Medio,
    ROUND(AVG(InitialPrice), 1) as Media_Preco_Inicial,
    ROUND(AVG(ActualPrice), 1) as Media_Preco_Atual
FROM schemas.gold.unified_games_analytics
GROUP BY ALL;

Analisando os resultados, pode-se questionar se eventualmente a data de lançamento do jogo pode impactar a análise, tendo em vista que jogos mais antigos tradicionalmente recebem descontos progressivos ao longo do tempo. Portanto, faz-se necessário um recorte periódico, aqui observamos os jogos somente a partir de XXXXX.

In [0]:
%sql
SELECT 
    CASE 
        WHEN Critic_Score >= 90 THEN '1. Aclamados (90-100)'
        WHEN Critic_Score BETWEEN 75 AND 89 THEN '2. Bons (75-89)'
        WHEN Critic_Score BETWEEN 60 AND 74 THEN '3. Medianos (60-74)'
        WHEN Critic_Score < 60 THEN '4. Fracos (<60)'
        ELSE 'N/A'
    END AS Faixa_Critica,
    COUNT(*) as Qtd_Jogos,
    ROUND(AVG(Discount), 1) as Desconto_Medio,
    ROUND(AVG(InitialPrice), 1) as Media_Preco_Inicial,
    ROUND(AVG(ActualPrice), 1) as Media_Preco_Atual
FROM schemas.gold.unified_games_analytics
WHERE Year_of_Release >= 2014
GROUP BY ALL
ORDER BY 1;

Agora, realizando um recorte para lançamentos realizados em 2014 e anos posteriores, levando em conta que a base possui dados até 2016, podemos ter uma maior assertividade na análise, de forma que fica evidente que os jogos com maior sucesso na crítica costumam ter descontos menores.

### 2. O sucesso das notas dos jogadores dita o tamanho do desconto? Jogos com maior nota possuem maior preço?

Agora, a fim de analisar se a nota dos jogadores possuem o mesmo impacto na manutenção dos preços por parte das publishers:

In [0]:
%sql
SELECT 
    CASE 
        WHEN User_Score >= 9 THEN '1. Aclamados (9-10)'
        WHEN User_Score BETWEEN 7.5 AND 9 THEN '2. Bons (7.5-9)'
        WHEN User_Score BETWEEN 6 AND 7 THEN '3. Medianos (6-7)'
        WHEN User_Score < 6 THEN '4. Fracos (<6)'
    END AS Faixa_Usuarios,
    COUNT(*) as Qtd_Jogos,
    ROUND(AVG(Discount), 1) as Desconto_Medio,
    ROUND(AVG(InitialPrice), 1) as Media_Preco_Inicial,
    ROUND(AVG(ActualPrice), 1) as Media_Preco_Atual
FROM schemas.gold.unified_games_analytics
WHERE User_Score IS NOT NULL 
    AND Discount IS NOT NULL
    AND (
        (User_Score >= 9)
        OR (User_Score BETWEEN 7.5 AND 9)
        OR (User_Score BETWEEN 6 AND 7)
        OR (User_Score < 6)
    )
    AND Year_of_Release >= 2014
GROUP BY ALL
ORDER BY 1;


Analisando os resultados oriundos da análise das notas dos usuários, percebe-se o mesmo padrão apresentado nas notas da crítica, onde os jogos com notas mais baixas costumam receber descontos maiores, bem como concentrar o maior volume de de ocorrências.

Já quando olhamos para o preço praticado, jogos com maiores avaliações costumam ser lançados com um preço mais elevado, enquanto jogos com avaliações maix baixa costuram ter um preço menor, orbitando uma faixa de 19 dólares. Entretanto para os jogadores, a faixa de preço dos jogos "Aclamados" é ligeiramente superior a faida de preço dos críticos, podendo indicar que a crítica costuma possuir um olhar mais imparcial para o custo x retorno da obra.

### 3. Quais gêneros possuem a melhor nota média?

In [0]:
%sql
SELECT 
    Genre,
    COUNT(*) as Qtd_Jogos,
    ROUND(AVG(Critic_Score), 1) as Media_Nota_Critica,
    ROUND(AVG(User_Score), 1) as Media_Nota_Usuario
FROM schemas.gold.unified_games_analytics
WHERE Genre IS NOT NULL AND Critic_Score IS NOT NULL
GROUP BY Genre
HAVING COUNT(*) > 10 -- Filtro nichos muito pequenos para não sujar a análise
ORDER BY Media_Nota_Critica DESC;

Quando se analisam gêneros e notas, percebe-se que os criticos costumam apresentar noas mais elevadas para jogos de luta e esportes, enquanto os jogadores costumam apresentar novas melhores para jogos de aventura, onde os críticos costumam ser ligeiramente mais duros.
### 4. Quais generos que possuem maior desconto?

In [0]:
%sql
SELECT 
    Genre,
    COUNT(*) as Qtd_Jogos,
    ROUND(AVG(Discount), 1) as Media_Desconto_Percentual,
    ROUND(AVG(ActualPrice), 2) as Preco_Final_Medio
FROM schemas.gold.unified_games_analytics
WHERE Genre IS NOT NULL 
  AND Discount > 0 -- Consideramos apenas jogos que têm algum desconto ou política de preço
GROUP BY Genre
ORDER BY Media_Desconto_Percentual DESC;

Quando analisamos os descontos aplicados por gêneros de jogos, percebe-se que jogos do estilo shooter e ação/action costumam ter um desconto maior quando levamos em consideração a quantidade de ocorrências.
### 5. A nota da crítica é muito diferente da nota dos jogadores?

In [0]:
%sql
SELECT 
    Genre,
    ROUND(AVG(Critic_Score), 1) as Media_Critica,
    ROUND(AVG(User_Score * 10), 1) as Media_Usuario_Norm, -- Normaliza para escala 0-100
    
    -- Calcula a diferença absoluta média
    ROUND(AVG(ABS(Critic_Score - (User_Score * 10))), 1) as Diferenca_Media
FROM schemas.gold.unified_games_analytics
WHERE Critic_Score IS NOT NULL AND User_Score IS NOT NULL
GROUP BY Genre
ORDER BY Diferenca_Media DESC; 

Como a análise anterior já apresentava, os crícitos costumam conceder notas maiores para os jogos de esportes e luta, e há uma coesão maior para jogos do gênero de ação e puzzle.

# Autoavaliação

O maior desafio técnico deste MVP foi a unificação de dados heterogêneos sem um identificador universal comum (Unique ID), como um código SKU ou ISBN. As bases de dados de origem (Vendas Físicas e Steam) utilizavam padrões de nomenclatura distintos, o que inviabilizou junções diretas simples.

Para superar isso, implementei na camada Silver uma estratégia de normalização sintática (utilizando funções LOWER e TRIM e remoção de caracteres especiais), permitindo criar uma chave de junção artificial, lonmge de ser a ideal, mas ainda funcional.

_Auto-crítica_: Reconheço que, embora essa abordagem resolva alguns dos casos com um "match exato", ela possui limitações em títulos com variações semânticas (ex: "The Witcher 3" vs "The Witcher III"). Em uma evolução futura (V2), seria necessária a implementação de algoritmos de _Fuzzy Matching_ ou o uso de uma tabela "de-para" manual para elevar a acurácia da unificação para próximo de 100%, mas até o momento o tratamento realizado permite a utilização de apenas 10% da base.

A despeito destes fatores, o MVP apresentado responde os pontos levantados ao começo do documento e com as premissas do projeto de forma geral.